In [ ]:
# Mout Google Drive
# https://towardsdatascience.com/google-drive-google-colab-github-dont-just-read-do-it-5554d5824228
from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)
# %pwd %ls
# run github settings
%run /content/drive/MyDrive/CNNStanford/pytorch/pytorch_sandbox/Colab_Helper.ipynb

In [ ]:
MESSAGE = "clean file & gitignore again"
!git config --global user.email "ronyginosar@mail.huji.ac.il"
!git config --global user.name "ronyginosar"
!git add .

In [ ]:
!git commit -m "{MESSAGE}"
!git push "{GIT_PATH}"

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


DATA_ROOT = "" # add sample DS

# (1) Define Dataset
class GenericImageFolder(Dataset):
    def __init__(self, root_dir):
        # Get the root dir of images
        self.root_dir = root_dir
        # Find for all jpeg images recursively  (e.g. using Pathlib & rglob)
        # images = []
        # p = Path(root_dir)
        # for i in p.rglob('*.jpg'): --> p.glob s already recursive...
        #     images.append(i.name)
        images = list(Path(root_dir).rglob("*.jpg"))
        self.images = images
        # used transform
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((256, 256))
        ])

    def __len__(self):
        return len(self.images)  # was missing

    def __getitem__(self, idx):
        # Loads an image (by idx) from disk
        im_path = Path(self.root_dir)/self.images[idx]
        image = Image.open(im_path)
        # (using transforms) Resize to 256x256 and convert to tensor  (transforms.Resize, ToTensor)
        # image = transforms.ToTensor()(image)
        # image = transforms.Resize((256, 256))
        # moved above to init + compose(sequence of transforms):
        image = self.transform(image)
        # Return the image as a float tensor 3x256x256 (range [0, 1])   (no GT)  -> built in to ToTransform
        return image


# a function that gets an image tensor and displays it to screen (using matplotlib)
def show_image(im_tensor):
    im = transforms.ToPILImage()(im_tensor.to('cpu'))
    plt.imshow(np.asarray(im))
    # or from previous ex1.2: plt.imshow(im_tensor.permute((1,2,0)).cpu().numpy())
    plt.show()  # needed for imshow


# (2) Create dataset
dataset = GenericImageFolder(DATA_ROOT)

# (3) Visualize a dataset sample
show_image(dataset[2])

# (4) Create dataloader with batch_size=8
dataloader = DataLoader(dataset, batch_size=8)
# sanity:
# print(len(dataset), len(dataloader))

# (5) Iterate through dataloader
mean = torch.Tensor([0, 0, 0])  # torch.zeros(3)
dataloaderIterator = iter(dataloader)  # need to init this outside of loop, otherwise we get the same batch.
for batch in dataloader:
    img_batch = next(dataloaderIterator)
    # sanity:
    # print(f"images: shape={img_batch.shape}, type={img_batch.dtype}")

    # (6) Calculate mean R,G,B values across all images and spatial locations
    # shape of images (batch_size, c, h, w)
    # c is the depth of the patches (since they are RGB, so c=3)
    # no need to break into rgb - it returns a 1*3 tensor that saves them as is
    batch_mean = img_batch.shape[0] * img_batch.mean([0, 2, 3])  # note: these dims are the ones reduced
    mean += batch_mean
    # sanity
    # print(batch_mean, img_batch.shape[0])
    # why do we have to *batchsize and then later /datasetsize?
    # since we mean over each batch and want to the divide by all items.
    # however - 1) last batch can be different size (hence can do it outside of loop)
    # 2) # of items is no longer datasetsize but some adaptation (smaller), according to how many items in each batch
    # batch.mean(dim=(0,2,3)) vs torch.mean(batch, (0,2,3)) - same

print(f'mean RGB = {mean / len(dataset)}')
